# Interactive fitting with Minuit

The following script is an illustration of fitting, where it is possible to alter the fitting parameters interactively.

Adapted from iMinuit documentation and examples: https://scikit-hep.org/iminuit/

### Authors: 
- Malthe Nordentoft (Niels Bohr Institute)
- Troels C. Petersen (Niels Bohr Institute)

### Date:    
- 09-11-2025 (latest update)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from iminuit import Minuit, cost

## ChiSquare fit to two Gaussians:

In [ ]:
# Generate the data, which consists of two overlapping Gaussians:
data = np.hstack([np.random.randn(1000)*1.1, np.random.randn(1000)*0.6+3.1])

# Bin the data in order to draw it:
Nbins = 100
xmin, xmax = -4.0, 6.0
binwidth = (xmax-xmin) / Nbins
freq, bins = np.histogram(data, bins = np.linspace(xmin, xmax, Nbins))
bins = (bins[1:] + bins[:-1])/2
freq_err = np.sqrt(freq)

# For ChiSquare fitting:
x = bins[freq > 0]
y = freq[freq>0]
sy = freq_err[freq >0]
print("Number of non-empty bins:", len(x))

In [ ]:
# Fitting functions:
def double_gaussian(x, N, mu1, mu2, sigma1, sigma2, f):
    return N * binwidth * (f*stats.norm.pdf(x, mu1, sigma1) + (1-f)*stats.norm.pdf(x, mu2, sigma2))

In [ ]:
# ChiSquare fit (of course binned):
c = cost.LeastSquares(x, y, sy, double_gaussian)

# Set the initial values (very important) and the allowed parameter ranges (for widget):
m = Minuit(c, N = 1000, mu1 = -2, mu2 = 4, sigma1 = 0.5, sigma2 = 1.0, f = 0.3)
m.limits["N"] = (0, 5000)
m.limits["mu1", "mu2"] = (-10, 10)
m.limits["sigma1", "sigma2"] = (0.01, 3)
m.limits["f"] = (0, 1)

In [ ]:
m.interactive()

### Notes to fit:

To begin with the <b>initial parameters are not very good</b>, and if you go ahead and press "fit" right away, then you get a poor result.<br>

However, if you instead change the <b>initial parameters to reasonable values</b>, the subsequent fit converges nicely.<br>

Given that this is a ChiSquare fit, you should ask yourself, if you are satisfied with the fit?

## ChiSquare and Unbinned Likelihood fit to Exponential background with Gaussian peak:

In [ ]:
gauss = np.random.randn(1000)*0.1 + 0.5
exp = stats.expon.rvs(size = 2000, scale = 1)
exp = exp[exp < 1]
data = np.append(gauss, exp)
truth = [len(gauss) / len(data), 0.8, 0.1, 1.0,1.0]

In [ ]:
def trunk_expo(x, lam):
    xmax = np.max(x)
    xmin = np.min(x)
    area = stats.expon.cdf(xmax, loc = 0, scale = lam) - stats.expon.cdf(xmin, loc = 0, scale = lam)   # To nomalize the PDF
    return stats.expon.pdf(x, loc = 0, scale = lam)/area

def model(x, N, f, mu, sigma, slope):
    return N * (f * stats.norm.pdf(x, mu, sigma) + (1 - f) * trunk_expo(x, slope))

c = cost.UnbinnedNLL(data, model)
m = Minuit(c, *truth)
m.fixed["N"] = 1      # Fixed parameter to controls the area of the PDF. Using chi2 methods we assume the pdf is normalized, thus the area = 1
m.limits["f", "mu"] = (0, 1)
m.limits["sigma", "slope"] = (0.01, None)

m.interactive(model_points=1000)

In [ ]:
freq, bins = np.histogram(data, np.arange(0, 1, 0.025))
bins = (bins[1:]+ bins[:-1])/2
sy = np.sqrt(freq[freq > 0])
x = bins[freq > 0]
y = freq[freq > 0]

bw = (x[1]-x[0])*len(data)     # Rough area under a unnormalized PDF (bin width time number of datapoints in the original dataset)
truth[-1] = bw

c = cost.LeastSquares(x, y, sy, model)
m = Minuit(c, *truth)
m.limits["f", "mu"] = (0, 1)
m.limits["sigma", "slope"] = (0, None)
m.limits["N"] = (0, bw*2)

m.interactive(model_points = 1000)    # Important to set model_points, as otherwise the vizualizer interpolates the data weird and plots the data wrong. Dont know why